In [2]:
import sys
sys.path.append('/Users/billystrickland/Documents/code/resonators')

import matplotlib.pyplot as plt
from shabanipy.labber import LabberData
import numpy as np
import os
from matplotlib.pyplot import cm
from utils.jy_mpl_settings.settings import jy_mpl_rc
from utils.jy_mpl_settings.colors import line_colors
from utils.shabanipy.labber import LabberData
import utils.resonator_functions as rf
import utils.fluxonium_functions as ff

In [5]:
from scipy.signal import savgol_filter

def smooth_array(data, window_length=51, polyorder=3):
    smoothed_data = savgol_filter(data, window_length, polyorder)
    return smoothed_data

def subtract_background(data, **kwargs):
    return data-smooth_array(data[50],**kwargs)

sample = 'JS801-4fluxonium@v5-01-CD02-WMSLJB'
file_num = '353'
gate, dummy, signal = ff.extract_data(f'{sample}-{file_num}.hdf5',
#                                       '03', '14',
                                   channels = ['Gate - Source voltage', 'Dummy - Value 2', 'VNA - S21'])

freq = np.unique(signal[0])
signal = rf.to_db(abs(signal[1]))
gate = np.unique(gate)

signal_b = signal[0]
signal_f = signal[1]
signal_b = subtract_background(signal_b, polyorder = 3)
signal_f = subtract_background(signal_f)

fig, ax = ff.plot_data(signal_b, 
            xlabel = '$f_\mathrm{probe}$ (GHz)', 
            ylabel = '$V_G$ (V)',
            zlabel = '$|S_21|$ (arb.)',
            ylim = [gate[0], gate[-1]],
            xlim=[freq[0]*1e-9, freq[-1]*1e-9],
            figsize = (7,5),
            zlim=[1, -3]
                   )

# array_of_arrays = signal_b[::-1]
# freqs_b = freq[find_troughs(array_of_arrays)]
# plt.plot(freqs_b, gate[0:90], linestyle = 'None', marker = '.')

# fig, ax = plot_data(signal_f[::-1], 
#             xlabel = '$f_\mathrm{probe}$ (GHz)', 
#             ylabel = '$V_G$ (V)',
#             lims = [freq[0], freq[-1], gate[0], gate[-1]],
                    
#             figsize = (12,5)
#                    )

# array_of_arrays = signal_f
# # freqs_f = freq[find_troughs(array_of_arrays)]
# # plt.plot(freqs_f, gate[0:90], linestyle = 'None', marker = '.')

# fig, ax = plt.subplots()
# # plt.plot(gate[0:90], freqs_f[::-1], linestyle = 'None', marker = '.', label = 'backward')
# # plt.plot(gate[0:90], freqs_b[::-1], linestyle = 'None', marker = '.', label = 'forward')
# plt.legend()
# plt.ylabel('$f_\mathrm{r}$ (GHz)')
# plt.xlabel('$V_G$ (V)')
ff.save_fig(fig, 'gate_hysteresis')

OSError: Unable to open file (unable to open file: name = 'JS801-4fluxonium@v5-01-CD02-WMSLJB-353.hdf5', errno = 2, error message = 'No such file or directory', flags = 0, o_flags = 0)

In [ ]:
def extract_data(sample, file_num, month, day, year = '2024', 
                 channels = ['Qcage - Magnet - Source voltage', 'VNA - S21', 'SC1 - Frequency'],
                 root = f'/Users/billystrickland/Library/CloudStorage/GoogleDrive-wms269@nyu.edu/.shortcut-targets-by-id/1p4A2foj_vBr4k6wrGEIpURdVkG-qAFgb/nyu-quantum-engineering-lab/labber/data-backups/qubitfridge/Data/'):
    
    FILE = f'{root}{year}/{month}/Data_{month}{day}/{sample}-{file_num}.hdf5'
    with LabberData(FILE) as f:
        data=[]
        for i in channels:
            data.append(f.get_data(i, get_x = True))
    return data

sample = 'JS801-4fluxonium@v5-01-CD02-WMSLJB'
file_num = '1026'
data = extract_data(sample, file_num, '04', '09', 
                    channels = [ 'SC3 - Drive - Frequency', 'Gate - Source voltage', 'VNA - S21'])
drive = data[0]
drive = np.unique(drive)
drive = drive[::-1]

gate = data[1]
gate = np.unique(gate)
gate = gate[::-1]

freq = data[2][0]
freq = np.unique(freq)

from_data = abs(data[2][1])

fig, ax = ff.plot_data(ff.normalize(from_data[:,0]),
            ylim=[gate[-1], gate[0]],
            xlim=[freq[0]*1e-9, freq[-1]*1e-9],
            xlabel='$f_\mathrm{probe}$ (GHz)',
            ylabel='$V_G$ (V)',
            figsize = (7, 5),
            zlim=[.4, .95]
            )
ff.save_fig(fig, 'single_tone_vs_gate')

signal_mag = ff.dynamic_slice(from_data, dispersive=0).T
signal_mag = ff.normalize(signal_mag[30::])

fig, ax = ff.plot_data(signal_mag[:, 70::],
            ylim=[gate[-1], gate[0]],
#             xlim=[drive[-1]*1e-9, drive[30]*1e-9],
            xlim=[drive[-30]*1e-9, drive[0]*1e-9],
            xlabel='$f_\mathrm{drive}$ (GHz)',
            ylabel='$V_G$ (V)',
            figsize = (7, 5),
            zlim = [-0.5, 3]
            )

ff.save_fig(fig, f'{sample}-{file_num}_two_tone_vs_gate')

In [ ]:
from scipy.signal import find_peaks

def find_peak_in_array(data, threshold=0.5, **kwargs):
    peaks_indices, _ = find_peaks(data, height=threshold, **kwargs)
    return peaks_indices

def clean_array(arr):
    cleaned_arr = np.zeros_like(arr)
    for i in range(len(arr)):
        for j in range(len(arr[i])):
            if isinstance(arr[i][j], np.ndarray):
                if arr[i][j].size == 0:
                    cleaned_arr[i][j] = 0
                else:
                    cleaned_arr[i][j] = arr[i][j][0]  # Take the first element
            else:
                cleaned_arr[i][j] = arr[i][j]
    return cleaned_arr


def find_pattern_around_number(arr, threshold=3):
    pattern_indices = []
    averages = []
    for i in range(len(arr)-2):
        # Check if three consecutive numbers are close enough to each other
        if abs(arr[i][2] - arr[i+1][2]) <= threshold and abs(arr[i+1][2] - arr[i+2][2]) <= threshold:
            pattern_indices.append(i)
            # Calculate average of the three numbers
            avg = (arr[i][2] + arr[i+1][2] + arr[i+2][2]) / 3
            averages.append(avg)
    return pattern_indices, averages


In [ ]:
gate_key = []
probe_freq_i_key = []
qubit_freq_key = []

In [ ]:
gate_i = 80
to_extract = qf.average_data(from_data[gate_i, :, :].T)
storage = []

for i, x in enumerate(to_extract):
#     plt.plot(x)
    peak_index = find_peak_in_array(x, threshold=0.001, width = 10)
    trough_index = find_peak_in_array(-x, threshold=0.001, width =10)
    storage.append([i, peak_index, trough_index])

cleaned_arr = clean_array(storage)

gate_key.append(gate[gate_i])
probe_freq_i_key.append(int(max(find_pattern_around_number(cleaned_arr)[1])))
qubit_freq_key.append(drive[int(max(find_pattern_around_number(cleaned_arr)[1]))])

print(f'[{gate[gate_i]}, {int(max(find_pattern_around_number(cleaned_arr)[1]))}, {drive[int(max(find_pattern_around_number(cleaned_arr)[1]))]}],')

In [ ]:
fq_array = np.array([[-8.84875, 51, 9388000000.0],
[-8.86, 46, 9448000000.0],
[-8.87125, 37, 9556000000.0],
[-8.8825, 21, 9843000000.0],
[-8.93875, 30, 9796000000.0],
[-8.96125, 92, 8896000000.0],
[-8.9725, 115, 8620000000.0],
[-8.98375, 120, 8560000000.0],
[-8.995000000000001, 126, 8488000000.0],
[-9.00625, 129, 8452000000.0],
[-9.0175, 128, 8464000000.0],
[-9.02875, 116, 8608000000.0],
[-9.040000000000001, 109, 8692000000.0],
[-9.05125, 97, 8836000000.0],
# [-9.0625, 27, 10000000000.0],
[-9.07375, 80, 9040000000.0],
[-9.085, 25, 9028000000.0],
[-9.09625, 26, 8872000000.0],
[-9.1075, 111, 8668000000.0],
[-9.11875, 137, 8356000000.0],
[-9.13, 145, 8260000000.0],
[-9.14125, 157, 8116000000.0],
[-9.16375, 0, 7846000000.0],
[-9.175, 158, 8104000000.0],
[-9.186250000000001, 211, 7468000000.0],
[-9.22, 0, 7840000000.0],
[-9.231250000000001, 160, 8080000000.0],
[-9.2425, 0, 7876000000],
[-9.2875, 0, 6844000000],
[-9.29875, 267, 6796000000.0],
[-9.31, 304, 6352000000.0],
[-9.321250000000001, 54, 6100000000],
[-9.3325, 331, 6028000000.0],
[-9.34375, 322, 6136000000.0],
[-9.355, 310, 6280000000.0],
[-9.36625, 297, 6436000000.0],
[-9.377500000000001, 296, 6448000000.0],
[-9.38875, 314, 6232000000.0],
[-9.4, 52, 5824000000]])

fig, ax = plt.subplots(figsize = (6, 4))
plt.plot(fq_array[:,0], fq_array[:,2]*1e-9, marker = 'o', linestyle = 'None')
plt.xlabel('$V_G$ (V)')
plt.ylabel('$f_Q$ (GHz)')
ax2 = ax.twinx()
ylims = ax.get_ylim()
ax2.set_ylim(((ylims[0]-.8)**2/8/.8, (ylims[1]-.8)**2/8/.8))